In [1]:
import json
import requests
import config

from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, expr
from pyspark.sql.functions import col, lit, explode
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType
from pyspark.sql.types import StructType, StructField, StringType, MapType, TimestampType

from pyspark.sql import SparkSession

In [2]:
# Create a Spark session with the Kafka package
spark = SparkSession.builder \
    .appName("MarketDataProcessor") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2") \
    .getOrCreate()


:: loading settings :: url = jar:file:/Users/csuftitan/Documents/Development/Adv%20DB%20Project/marketDataAnalysis/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/csuftitan/.ivy2/cache
The jars for the packages stored in: /Users/csuftitan/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2e42ebb2-e7e2-4962-b4c7-069373c666bf;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.2 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 342ms :: artifacts dl 11ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.8-1 from central in [default]
	org.apache.commons#commons-pool2;2.6.2 from

In [ ]:
# # Kafka configuration for Spark
# kafka_params = {
#     "kafka.bootstrap.servers": "localhost:9092",
#     "subscribe": "marketdata"
# }

# # Define the schema for the incoming JSON data
# # Adjusted schema definition to match the JSON structure
# schema = StructType([
#     StructField("Meta Data", StructType([
#         StructField("1. Information", StringType(), True),
#         StructField("2. Symbol", StringType(), True),
#         StructField("3. Last Refreshed", TimestampType(), True),
#         StructField("4. Interval", StringType(), True),
#         StructField("5. Output Size", StringType(), True),
#         StructField("6. Time Zone", StringType(), True),
#     ]), True),
#     StructField("Time Series (5min)", MapType(
#         StringType(),
#         StructType([
#             StructField("1. open", StringType(), True),
#             StructField("2. high", StringType(), True),
#             StructField("3. low", StringType(), True),
#             StructField("4. close", StringType(), True),
#             StructField("5. volume", StringType(), True),
#         ])
#     ), True),
# ])

# # Read data from Kafka
# df = spark.readStream.format("kafka").options(**kafka_params).load().select(from_json(col("value").cast("string"), schema).alias("parsed_value"))

# # # Create a PySpark DataFrame
# # df = spark.createDataFrame([(df[""])], schema=schema)
# # print("=-=======================")
# # df.head()

# # df = df.select("parsed_value.Meta Data.`2. Symbol`","parsed_value.Time Series (5min).`1. open`") #, 'parsed_value.Time Series (5min).2. high','parsed_value.Time Series (5min).3. low','parsed_value.Time Series (5min).4. close', 'parsed_value.Time Series (5min).5. value')

# # df = spark.createDataFrame([(df["parsed_value.Meta Data"]["`2. Symbol`"])], schema=schema)

# # df.show()
# # Start running the query that prints the running counts to the console
# # query = df\
# #     .writeStream\
# #     .outputMode('append')\
# #     .option("truncate", "false")\
# #     .format('console')\
# #     .start()\
# #     .awaitTermination()

# query = df \
#     .writeStream \
#     .format("memory") \
#     .queryName("my_table") \
#     .start()
# # Wait for the streaming query to start
# query.awaitTermination()

# # Query the in-memory table using Spark SQL to create a static DataFrame
# static_df = spark.sql("SELECT * FROM my_table")

# # Show the resulting static DataFrame
# static_df.show(truncate=False)


In [ ]:
# Kafka configuration for Spark
# kafka_params = {
#     "kafka.bootstrap.servers": "localhost:9092",
#     "subscribe": "marketdata"
# }


# schema = (
#         StructType()
#         .add("symbol", StringType())
#         .add("timeseries", StringType())
#         .add("close", StringType())
#     )

# df = spark\
#       .readStream \
#       .format("kafka") \
#       .option("kafka.bootstrap.servers", "localhost:9092") \
#       .option("subscribe", "marketdata") \
#       .load()
      

# query = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
#     .writeStream \
#     .format("console") \
#     .option("truncate", "false")\
#     .start()

# query.awaitTermination()
# Define the schema for the incoming JSON data
# Adjusted schema definition to match the JSON structure


# Read data from Kafka into a DataFrame
# kafka_stream_df = spark \
#     .readStream \
#     .format("kafka") \
#     .options(**kafka_params) \
#     .load() \
#     .selectExpr("CAST(value AS STRING)") \
#    .select(from_json("value", schema).alias("json_data")) \
#     .select("json_data.*")

# kafka_stream_df = spark \
#     .readStream \
#     .format("kafka") \
#     .options(**kafka_params) \
#     .load() \
#     .select(["*"])

# Perform any additional transformations as needed
# kafka_stream_df.printSchema()
# # Output the DataFrame to the console for testing
# query = kafka_stream_df \
#     .writeStream \
#     .outputMode("append") \
#     .format("console") \
#     .start()

# writing_df = kafka_stream_df.writeStream \
#     .format("json") \
#     .option("path", "data/output/device_data") \
#     .option("checkpointLocation","checkpoint_dir") \
#     .outputMode("append") \
#     .start()

# Wait for the streaming query to finish
# query.awaitTermination()



In [6]:
kafka_params = {
    "kafka.bootstrap.servers": "localhost:9092",
    "subscribe": "marketdata"
}


# Define the schema for the "value" field
value_schema = StructType([
    StructField("symbol", StringType(), True),
    StructField("timeseries", StringType(), True),
    StructField("close", StringType(), True),
])

# Read data from Kafka topic
kafka_df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "marketdata") \
    .load()


kafka_df = kafka_df.select(from_json(col("value").cast("string"), value_schema) \
                        .alias("values"))

from pyspark.sql.functions import explode


parsed_df = kafka_df.select("values.*")

# qry = parsed_df.writeStream.outputMode("append").format("console").start()

# qry.awaitTermination()

# Write the streaming DataFrame to an in-memory table
query = parsed_df \
    .writeStream \
    .format("memory") \
    .queryName("my_table3") \
    .start()

# Wait for the streaming query to start
query.awaitTermination(10)

# Query the in-memory table using Spark SQL to create a static DataFrame
static_df = spark.sql("SELECT * FROM my_table3")

# Show the resulting static DataFrame
# static_df.show(truncate=False)
static_df.printSchema()

from pyspark.sql.functions import split, expr
import pyspark.sql.functions as F
from pyspark.sql.types import *

df = static_df.withColumn("time", expr("split(trim('[]', timeseries), ',' )"))
df = df.withColumn("value", expr("split(trim('[]', close), ',' )"))

df = df.withColumn("new", F.arrays_zip("time", "value"))\
       .withColumn("new", F.explode("new"))\
       .select("symbol", F.col("new.time").alias("timeseries"), F.col("new.value").alias("close"))


df.show(truncate=False)



23/11/29 03:17:50 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/3h/dgk1gddd4vd7jthqz32mnkhh0000gn/T/temporary-1d2d1d23-4800-4ae6-8362-caaca56eafb7. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/11/29 03:17:50 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
23/11/29 03:17:50 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
23/11/29 03:17:50 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
23/11/29 03:17:50 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
23/11/29 03:17:50 WARN AdminClientConfig: The configuration '

root
 |-- symbol: string (nullable = true)
 |-- timeseries: string (nullable = true)
 |-- close: string (nullable = true)

+------+---------------------+----------+
|symbol|timeseries           |close     |
+------+---------------------+----------+
|AAPL  |"2023-11-28 19:55:00"|"189.9400"|
|AAPL  |"2023-11-28 19:50:00"|"189.8350"|
|AAPL  |"2023-11-28 19:45:00"|"189.6600"|
|AAPL  |"2023-11-28 19:40:00"|"189.6500"|
|AAPL  |"2023-11-28 19:35:00"|"189.6500"|
|AAPL  |"2023-11-28 19:30:00"|"189.6550"|
|AAPL  |"2023-11-28 19:25:00"|"189.6400"|
|AAPL  |"2023-11-28 19:20:00"|"189.7400"|
|AAPL  |"2023-11-28 19:15:00"|"189.6400"|
|AAPL  |"2023-11-28 19:10:00"|"189.6400"|
|AAPL  |"2023-11-28 19:05:00"|"189.6900"|
|AAPL  |"2023-11-28 19:00:00"|"189.7700"|
|AAPL  |"2023-11-28 18:55:00"|"189.7420"|
|AAPL  |"2023-11-28 18:50:00"|"189.7700"|
|AAPL  |"2023-11-28 18:45:00"|"189.7800"|
|AAPL  |"2023-11-28 18:40:00"|"189.7900"|
|AAPL  |"2023-11-28 18:35:00"|"189.7500"|
|AAPL  |"2023-11-28 18:30:00"|"189.65